# SETUP

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unicodedata import normalize
import re
import pycountry
from countryinfo import CountryInfo
from currencies import Currency
import requests
import json
from bs4 import BeautifulSoup

# Airline and country

In [2]:
url = "https://www.pilotjobsnetwork.com/"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

country_airlines = []

elements = soup.find_all("a", href=lambda href: href and "jobs" in href)
for element in elements :
    text = element.text.strip()
    country_airlines.append(text)

# country_airlines

In [3]:
airlines_by_country = []
for text in country_airlines:
    # Regex pattern to match the country and airline
    apattern = r'^[^-]*-[^-]*-\s*([^(]+)'
    cpattern = r'(\w+)\s\((\w{2})\)'
    

    country_match = re.search(cpattern, text)
    country = country_match.group(1)
    airline_match = re.search(apattern, text)
    airline = airline_match.group(1)

    # Append the airline and country to the list
    airlines_by_country.append((airline, country))

# Create the table
df = pd.DataFrame(airlines_by_country, columns=['Airline', 'Country'])
df['Country'] = df['Country'].replace('UK', 'United Kingdom')
# # Print the table
# df.head(10)

In [4]:
# Define countries standard name (ISO)
# Define a function to map country names to their ISO codes
def map_country_name(name):
    try:
        return pycountry.countries.search_fuzzy(name)[0].name
    except LookupError:
        return None
# Add a column to the dataframe with the ISO code of the country
df['ISO'] = df['Country'].apply(map_country_name)

In [5]:
df_airline = df

In [6]:
df.to_csv("metadata_ctry_airlines.csv")

# Average gross income by country and Currency code (frozen)

In [7]:
url_3 = "https://www.worlddata.info/average-income.php"
df_3 = pd.read_html(url_3, attrs={'class': 'std100 hover', 'id': 'tabsort'})
df_3 = df_3[0]
df_3['Country'] = df_3['Country'].str.replace(' \*', '', regex=True)

In [8]:
# Define countries standard name (ISO)
# Define a function to map country names to their ISO codes
def map_country_name(name):
    try:
        return pycountry.countries.search_fuzzy(name)[0].name
    except LookupError:
        return None
# Add a column to the dataframe with the ISO code of the country
df_3['ISO'] = df_3['Country'].apply(map_country_name)

In [9]:
# Define a function to get the ISO code for each country
def get_iso_code(country_name):
    try:
        country = pycountry.countries.search_fuzzy(country_name)[0]
        return country.alpha_3
    except LookupError:
        return None
df_3['ISO_code'] = df_3['ISO'].apply(get_iso_code)

In [10]:
def get_currency(country):
    try:
        currency = CountryInfo(country).currencies()
        return currency
    except LookupError:
        return None
df_3['Currency'] = df_3['ISO'].apply(get_currency)

def first_item_to_string(arr):
    return str(arr[0])
# df_3['Currency'] = df_3['Currency'].apply(first_item_to_string)

In [11]:
df_3.columns = ['Rank', 'Country', 'yearly_income', 'monthly_income', 'ISO', 'ISO_code', 'Currency']

In [12]:
df_3["yearly_income"] = df_3["yearly_income"].str.replace(",", "").str.replace("$", "").astype(float)
df_3["monthly_income"] = df_3["monthly_income"].str.replace(",", "").str.replace("$", "").astype(float)
df_3["Currency"] = df_3["Currency"].str.replace("[',\[\]]", "")

/var/folders/y9/x007ynb92t57yt7s517ct53w0000gn/T/ipykernel_41258/68737043.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_3["yearly_income"] = df_3["yearly_income"].str.replace(",", "").str.replace("$", "").astype(float)
/var/folders/y9/x007ynb92t57yt7s517ct53w0000gn/T/ipykernel_41258/68737043.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_3["monthly_income"] = df_3["monthly_income"].str.replace(",", "").str.replace("$", "").astype(float)
/var/folders/y9/x007ynb92t57yt7s517ct53w0000gn/T/ipykernel_41258/68737043.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_3["Currency"] = df_3["Currency"].str.replace("[

In [13]:
df_income_code_cur = df_3

In [14]:
df_3.to_csv("metadata_ctry_salary_2.csv")

# Currency Exchange rates

In [15]:
df_V1 = pd.read_csv("METADATA_V1.csv")
df_V1

,Country,year_income_gross,month_income_gross,ISO,Currency
0,Monaco,186080,15507,Monaco,EUR
1,Bermuda,122470,10206,Bermuda,BMD
2,Switzerland,90600,7550,Switzerland,CHF
3,Luxembourg,88190,7349,Luxembourg,EUR
4,Norway,83880,6990,Norway,NOK
...,...,...,...,...,...
64,Pakistan,1470,123,Pakistan,PKR
65,Nepal,1220,102,Nepal,NPR
66,Timor-Leste,1140,95,Timor-Leste,LCU
67,Ethiopia,940,78,Ethiopia,ETB


In [16]:
df_V1.to_json("ctry_salry.json", orient="records")

In [17]:
payload = {}
headers= {
  "apikey": "pPf6uwtTqHvmPDaWzHFzihcpKRAK4SC5"}
from_usd = "USD"
amount_convert = 1
def convert_usd(code_currency):
    try:
        response = requests.request("GET", f"https://api.apilayer.com/exchangerates_data/convert?to={code_currency}&from={from_usd}&amount={amount_convert}", headers=headers, data = payload)
        return json.loads(response.text)["info"]["rate"]
    except LookupError:
        return None

df_V1['XR'] = df_V1['Currency'].apply(convert_usd)

In [18]:
df_V2 = df_V1[["Currency", "XR"]]
df_V2 = df_V2.drop_duplicates()

In [19]:
df_XR = df_V2

In [20]:
df_V2.to_csv("XR_list.csv")


# Index cost of living (frozen)

In [21]:
url_4 = "https://www.numbeo.com/cost-of-living/rankings_by_country.jsp"
df_4 = pd.read_html(url_4, attrs={'id': 't2'})
df_4 = df_4[0]

In [22]:
# Define a function to map country names to their ISO codes
def map_country_name(name):
    try:
        return pycountry.countries.search_fuzzy(name)[0].name
    except LookupError:
        return name
# Add a column to the dataframe with the ISO code of the country
df_4['ISO'] = df_4['Country'].apply(map_country_name)

df_4= df_4.replace("Hong Kong (China)", "Hong Kong")
df_4= df_4.replace("Macao (China)", "Macao")
df_4= df_4.replace("Taiwan (China)", "Taiwan")
df_4= df_4.replace("Kosovo (Disputed Territory)", "Kosovo")
df_4 = df_4[['ISO','Cost of Living Index']]
df_4 = df_4.dropna(how='any',axis=0) 

In [23]:
df_index = df_4

In [24]:
df_4.to_csv("cost_of_living_index.csv")

# Average salary income tax (frozen)

In [25]:
df_5 = pd.read_csv("salary_tax.csv")
# Define a function to map country names to their ISO codes
def map_country_name(name):
    try:
        return pycountry.countries.search_fuzzy(name)[0].name
    except LookupError:
        return name
# Add a column to the dataframe with the ISO code of the country
df_5['ISO'] = df_5['Country'].apply(map_country_name)
df_5["Last"].astype(float)
df_5.columns = ["Country", "Income_tax", "ISO"]

In [26]:
df_tax = df_5

In [27]:
df_5.to_csv("income_tax_updated.csv")

# Inflation rate (frozen)

In [28]:
#Load inflation xls and add ISO countries
df_V3 = pd.read_csv("SQ_pred_database - 7 - Inflation.csv")
df_V3 

# Define a function to get the ISO name for each country
def get_iso_code(name):
    try:
        return pycountry.countries.search_fuzzy(name)[0].name
    except LookupError:
        return None
df_V3['ISO_name'] = df_V3['Country'].apply(get_iso_code)
df_inflation = df_V3
df_inflation

,ISO_code,Country,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,ISO_name
0,ABW,Aruba,"2,08","4,32","0,63","-2,37","0,42","0,48","-0,89","-0,47","3,58","4,26","1,22","0,74","6,04",Aruba
1,AFG,Afghanistan,"2,18","11,8","6,44","7,39","4,67","-0,66","4,38","4,98","0,63","2,3","5,44","5,06",NaN,Afghanistan
2,AGO,Angola,"14,48","13,48","10,28","8,78","7,3","9,16","32,38","29,84","19,63","17,08","21,02","23,85","21,35",Angola
3,ALB,Albania,"3,57","3,41","2,03","1,93","1,62","1,91","1,29","1,99","2,03","1,41","1,62","2,04","6,73",Albania
4,ARE,United Arab Emirates,"0,88","0,88","0,66","1,09","2,34","4,07","1,62","1,97","3,06","-1,93","-2,08","0,18","5,22",United Arab Emirates
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,WSM,Samoa,"-0,2","2,88","6,19","-0,21","-1,24","1,92","0,13","1,3","4,2","2,19","-1,57","-3,01","8,75",Samoa
189,YEM,"Yemen, Rep.","11,17","19,54","9,89","10,97","8,1","39,4",5,"30,4","27,6",10,"26,18","45,7","43,85",None
190,ZAF,South Africa,"4,06","5,02","5,72","5,78","6,14","4,51","6,59","5,27","4,5","4,13","3,22","4,61",7,South Africa
191,ZMB,Zambia,"8,5","6,44","6,59","6,99","7,82","10,11","17,86","6,58","7,49","9,8","15,73","22,02","10,99",Zambia


In [29]:
df_inflation.to_csv("inlfation_rate_ISO.csv")

# Category

In [30]:
from tqdm.contrib.concurrent import thread_map
pd.set_option('display.max_colwidth', None)

#GET AIRLINES URLS
url = 'https://www.pilotjobsnetwork.com/'
max_urls = 5

def get_airlines_urls(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.find_all('a')
    for link in links:
        href = link.get('href')
        if href and 'jobs/' in href:
            airline_url = url + href
            yield airline_url
##GET INFO FROM PAGES
def get_airline_tables(airline_url):
    try:
        response = requests.get(airline_url)
        # Parse the HTML content of the response using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        # Find all text that contains the phrase "pilot jobs --->"
        matching_text = soup.find_all(text=lambda text: text and "pilot jobs --->" in text)
        # Regular expression to match text between parentheses
        for text in matching_text:
            paren_regex = r'\((.*?)\)'
            categories = []
            for text in matching_text:
                categories += re.findall(paren_regex, text)
            return [airline_url] + categories
    except NameError:
        return airline_url

# MAP IT 
list_category = list(thread_map(get_airline_tables, get_airlines_urls(url)))

#DF AS OUTPUT
df_category = pd.DataFrame([category for category in list_category if type(category) == list])

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
0it [00:00, ?it/s]/var/folders/y9/x007ynb92t57yt7s517ct53w0000gn/T/ipykernel_41258/4040292485.py:24: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  matching_text = soup.find_all(text=lambda text: text and "pilot jobs --->" in text)
501it [00:44, 11.20it/s]


In [31]:
# Define the regex pattern
regex_pattern = r'[^/]+(?=/?$)'

# Define a function to apply the regex pattern and replace underscores with spaces
def clean_url(url):
    match = re.search(regex_pattern, url)
    cleaned_url = match.group(0).replace('_', ' ')
    return cleaned_url
df_category[0] = df_category[0].apply(clean_url)
df_category.columns = ["Airline", "Category"]
df_category

,Airline,Category
0,ITA Airways,Major/National/Low Cost
1,Air Hamburg Luftverkehrsgesellschaft mbH,Fractional/Corporate
2,Kuwait Airways,Major/National/Low Cost
3,Hifly,Charter
4,DAS Private Jets GmbH,Fractional/Corporate
...,...,...
495,Ellinair,Regional
496,Titan Airways Ltd,Regional
497,Aeropartner,Fractional/Corporate
498,Airnorth,Regional


In [32]:
df_category.to_csv("category.csv")

# Conso df (to train model)

## Loading PPJN data scraping

In [33]:
df_extract_ppjn = pd.read_csv("PPJN_extract_salary.csv")
df_extract_ppjn.head()

#fill in blank with gross + remove Nan salary + convert salary to numeric
df_extract_ppjn["gross-net"] = df_extract_ppjn["gross-net"].fillna("gross")
df_extract_ppjn = df_extract_ppjn.dropna()
df_extract_ppjn["salary"] = df_extract_ppjn["salary"].str.replace(',', '.')
df_extract_ppjn["salary"] = pd.to_numeric(df_extract_ppjn["salary"])
df_extract_ppjn["Year"] = df_extract_ppjn["Year"].astype(int)

## Mapping with metadata

In [34]:
#Create df with all features
df_conso = df_extract_ppjn[["Airline", "Year", "Top/Base", "salary", "currency", "gross-net"]]

#Add category
cat_map = dict(zip(df_category['Airline'], df_category['Category']))
df_conso["Category"] = df_conso["Airline"].map(cat_map)

#Add country
country_map = dict(zip(df_airline['Airline'], df_airline['ISO']))
df_conso["Country"] = df_conso["Airline"].map(country_map)

#Add income
income_map = dict(zip(df_income_code_cur["ISO"], df_income_code_cur["yearly_income"]))
df_conso["Income"] = df_conso["Country"].map(income_map)

#Add cost of living
index_map = dict(zip(df_index["ISO"], df_index["Cost of Living Index"]))
df_conso["Index"] = df_conso["Country"].map(index_map)


/var/folders/y9/x007ynb92t57yt7s517ct53w0000gn/T/ipykernel_41258/240251770.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_conso["Category"] = df_conso["Airline"].map(cat_map)


## Standardize salary 

In [73]:
df_inflation = pd.read_csv("inflation_amended.csv")
for x in range(2010, 2023):
    df_inflation[f"{x}"] = df_inflation[f"{x}"].str.replace(',', '.').astype(float)
    df_inflation.fillna(0, inplace=True)
    df_inflation[f"{x}"] = df_inflation[f"{x}"].replace(np.nan,0)
    df_inflation[f"{x}"] = df_inflation[f"{x}"]/100+1
    df_inflation["2023"] = 1
# df_inflation = df_inflation.set_index("ISO_name")

def update_year(year):
    if year < 2010:
        return 2010
    else:
        return year

df_conso['Year'] = df_conso['Year'].apply(update_year)

In [74]:
df_inflation["Cum2010"]= df_inflation["2011"]*df_inflation["2012"]*df_inflation["2013"]*df_inflation["2014"]*df_inflation["2015"]*df_inflation["2016"]*df_inflation["2017"]*df_inflation["2018"]*df_inflation["2019"]*df_inflation["2020"]*df_inflation["2021"]*df_inflation["2022"]*df_inflation["2023"]
df_inflation["Cum2011"]= df_inflation["2012"]*df_inflation["2013"]*df_inflation["2014"]*df_inflation["2015"]*df_inflation["2016"]*df_inflation["2017"]*df_inflation["2018"]*df_inflation["2019"]*df_inflation["2020"]*df_inflation["2021"]*df_inflation["2022"]*df_inflation["2023"]
df_inflation["Cum2012"]= df_inflation["2013"]*df_inflation["2014"]*df_inflation["2015"]*df_inflation["2016"]*df_inflation["2017"]*df_inflation["2018"]*df_inflation["2019"]*df_inflation["2020"]*df_inflation["2021"]*df_inflation["2022"]*df_inflation["2023"]
df_inflation["Cum2013"]= df_inflation["2014"]*df_inflation["2015"]*df_inflation["2016"]*df_inflation["2017"]*df_inflation["2018"]*df_inflation["2019"]*df_inflation["2020"]*df_inflation["2021"]*df_inflation["2022"]*df_inflation["2023"]
df_inflation["Cum2014"]= df_inflation["2015"]*df_inflation["2016"]*df_inflation["2017"]*df_inflation["2018"]*df_inflation["2019"]*df_inflation["2020"]*df_inflation["2021"]*df_inflation["2022"]*df_inflation["2023"]
df_inflation["Cum2015"]= df_inflation["2016"]*df_inflation["2017"]*df_inflation["2018"]*df_inflation["2019"]*df_inflation["2020"]*df_inflation["2021"]*df_inflation["2022"]*df_inflation["2023"]
df_inflation["Cum2016"]= df_inflation["2017"]*df_inflation["2018"]*df_inflation["2019"]*df_inflation["2020"]*df_inflation["2021"]*df_inflation["2022"]*df_inflation["2023"]
df_inflation["Cum2017"]= df_inflation["2018"]*df_inflation["2019"]*df_inflation["2020"]*df_inflation["2021"]*df_inflation["2022"]*df_inflation["2023"]
df_inflation["Cum2018"]= df_inflation["2019"]*df_inflation["2020"]*df_inflation["2021"]*df_inflation["2022"]*df_inflation["2023"]
df_inflation["Cum2019"]= df_inflation["2020"]*df_inflation["2021"]*df_inflation["2022"]*df_inflation["2023"]
df_inflation["Cum2020"]= df_inflation["2021"]*df_inflation["2022"]*df_inflation["2023"]
df_inflation["Cum2021"]= df_inflation["2022"]*df_inflation["2023"]
df_inflation["Cum2022"]= df_inflation["2023"]
df_inflation["Cum2023"]= 1



In [75]:
df_inflation

,ISO_code,Country,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,ISO_name,2023,Cum2010,Cum2011,Cum2012,Cum2013,Cum2014,Cum2015,Cum2016,Cum2017,Cum2018,Cum2019,Cum2020,Cum2021,Cum2022,Cum2023
0,ABW,Aruba,1.0208,1.0432,1.0063,0.9763,1.0042,1.0048,0.9911,0.9953,1.0358,1.0426,1.0122,1.0074,1.0604,Aruba,1,1.191191,1.141862,1.134714,1.162259,1.157398,1.151869,1.162213,1.167701,1.127342,1.081280,1.068247,1.0604,1,1
1,AFG,Afghanistan,1.0218,1.1180,1.0644,1.0739,1.0467,0.9934,1.0438,1.0498,1.0063,1.0230,1.0544,1.0506,1.0000,Afghanistan,1,1.660453,1.485200,1.395340,1.299320,1.241349,1.249596,1.197161,1.140370,1.133231,1.107753,1.050600,1.0000,1,1
2,AGO,Angola,1.1448,1.1348,1.1028,1.0878,1.0730,1.0916,1.3238,1.2984,1.1963,1.1708,1.2102,1.2385,1.2135,Angola,1,6.981923,6.152558,5.579033,5.128731,4.779805,4.378715,3.307686,2.547509,2.129490,1.818833,1.502920,1.2135,1,1
3,ALB,Albania,1.0357,1.0341,1.0203,1.0193,1.0162,1.0191,1.0129,1.0199,1.0203,1.0141,1.0162,1.0204,1.0673,Albania,1,1.317521,1.274075,1.248726,1.225081,1.205552,1.182957,1.167891,1.145104,1.122321,1.106716,1.089073,1.0673,1,1
4,ARE,United Arab Emirates,1.0088,1.0088,1.0066,1.0109,1.0234,1.0407,1.0162,1.0197,1.0306,0.9807,0.9792,1.0018,1.0522,United Arab Emirates,1,1.181870,1.171561,1.163879,1.151329,1.125004,1.081007,1.063774,1.043223,1.012248,1.032169,1.054094,1.0522,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,WSM,Samoa,0.9980,1.0288,1.0619,0.9979,0.9876,1.0192,1.0013,1.0130,1.0420,1.0219,0.9843,0.9699,1.0875,Samoa,1,1.230483,1.196037,1.126318,1.128688,1.142860,1.121330,1.119874,1.105503,1.060943,1.038206,1.054766,1.0875,1,1
189,YEM,"Yemen, Rep.",1.1117,1.1954,1.0989,1.1097,1.0810,1.3940,1.0500,1.3040,1.2760,1.1000,1.2618,1.4570,1.4385,Yemen,1,11.164407,9.339474,8.498930,7.658763,7.084888,5.082416,4.840396,3.711960,2.909060,2.644600,2.095894,1.4385,1,1
190,ZAF,South Africa,1.0406,1.0502,1.0572,1.0578,1.0614,1.0451,1.0659,1.0527,1.0450,1.0413,1.0322,1.0461,1.0700,South Africa,1,1.837823,1.749975,1.655292,1.564844,1.474321,1.410698,1.323481,1.257225,1.203086,1.155369,1.119327,1.0700,1,1
191,ZMB,Zambia,1.0850,1.0644,1.0659,1.0699,1.0782,1.1011,1.1786,1.0658,1.0749,1.0980,1.1573,1.2202,1.1099,Zambia,1,3.348606,3.146003,2.951499,2.758668,2.558587,2.323664,1.971546,1.849827,1.720930,1.567331,1.354300,1.1099,1,1


In [76]:
dog_rows = df_inflation[df_inflation['ISO_code'] == 'HKG']
dog_rows.to_excel("HKGinflation.xlsx")

pd.set_option('display.max_columns', 500)
dog_rows.head(None)

,ISO_code,Country,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,ISO_name,2023,Cum2010,Cum2011,Cum2012,Cum2013,Cum2014,Cum2015,Cum2016,Cum2017,Cum2018,Cum2019,Cum2020,Cum2021,Cum2022,Cum2023
73,HKG,"Hong Kong SAR, China",1.0231,1.0528,1.0406,1.0432,1.0444,1.03,1.0242,1.0148,1.0241,1.0287,1.0033,1.0157,1.0188,Hong Kong,1,1.3976,1.327508,1.275714,1.222885,1.170897,1.136793,1.109933,1.093746,1.068007,1.03821,1.034795,1.0188,1,1


In [77]:
# Standardize the salary in USD/GROSS/YEARLY/EST.2023

#rate
xr_dict = dict(zip(df_XR.Currency, df_XR.XR))
df_conso['new_salary_USD'] = df_conso.apply(lambda x: x['salary'] / xr_dict[x['currency']] if x['currency'] != 'USD' else x['salary'], axis=1)
#gross
tax_dict = dict(zip(df_tax.ISO, df_tax.Income_tax))
df_conso['new_salary_USD_GROSS'] = df_conso.apply(lambda x: x['new_salary_USD'] * (1+tax_dict[x['Country']]/100) if x['gross-net'] == 'net' else x['new_salary_USD'], axis=1)
#inflation
def calculate_new_salary(row):
    country = row['Country']
    year = row['Year']
    inflation_rate = df_inflation.loc[df_inflation["ISO_name"] == country, f"Cum{year}"].values
    if len(inflation_rate) == 0:
        # handle case where inflation rate is not found
        return None
    else:
        return row['new_salary_USD_GROSS'] * inflation_rate[0]

        

df_conso["new_salary_USD_GROSS_2023"] = df_conso.apply(calculate_new_salary, axis=1)

In [78]:
pd.set_option('display.max_columns', 100)
df_conso

,Airline,Year,Top/Base,salary,currency,gross-net,Category,Country,Income,Index,new_salary_USD,new_salary_USD_GROSS,new_salary_USD_GROSS_2023
0,British Airways Mainline,2023,CaptMax,181477.5,GBP,gross,Legacy,United Kingdom,44480.0,61.5,220676.216302,220676.216302,220676.216302
1,British Airways Mainline,2023,CaptMin,87876.0,GBP,gross,Legacy,United Kingdom,44480.0,61.5,106857.010835,106857.010835,106857.010835
2,Cargo Air,2019,CaptMax,48000.0,EUR,net,Cargo,Bulgaria,NaN,40.5,51512.094611,56663.304072,68294.837584
3,Cargo Air,2019,CaptMin,42000.0,EUR,net,Cargo,Bulgaria,NaN,40.5,45073.082784,49580.391063,59757.982886
4,DHL Air UK,2023,CaptMax,172102.0,GBP,gross,Cargo,United Kingdom,44480.0,61.5,209275.630191,209275.630191,209275.630191
5,DHL Air UK,2023,CaptMin,121646.0,GBP,gross,Cargo,United Kingdom,44480.0,61.5,147921.251991,147921.251991,147921.251991
6,LOT Polish Airlines,2022,CaptMax,293160.0,PLN,gross,Major/National/Low Cost,Poland,16850.0,38.6,66845.127073,66845.127073,66845.127073
7,LOT Polish Airlines,2022,CaptMin,264120.0,PLN,gross,Major/National/Low Cost,Poland,16850.0,38.6,60223.546741,60223.546741,60223.546741
10,Eurowings GmbH,2023,CaptMax,189021.0,EUR,gross,Major/National/Low Cost,Germany,51660.0,62.9,202851.409070,202851.409070,202851.409070
11,Eurowings GmbH,2023,CaptMin,111948.0,EUR,gross,Major/National/Low Cost,Germany,51660.0,62.9,120139.082655,120139.082655,120139.082655


In [79]:
df_model = df_conso[["Airline", "Top/Base", "Category", "Country", "Income", "Index", "new_salary_USD_GROSS_2023"]]
df_model

,Airline,Top/Base,Category,Country,Income,Index,new_salary_USD_GROSS_2023
0,British Airways Mainline,CaptMax,Legacy,United Kingdom,44480.0,61.5,220676.216302
1,British Airways Mainline,CaptMin,Legacy,United Kingdom,44480.0,61.5,106857.010835
2,Cargo Air,CaptMax,Cargo,Bulgaria,NaN,40.5,68294.837584
3,Cargo Air,CaptMin,Cargo,Bulgaria,NaN,40.5,59757.982886
4,DHL Air UK,CaptMax,Cargo,United Kingdom,44480.0,61.5,209275.630191
5,DHL Air UK,CaptMin,Cargo,United Kingdom,44480.0,61.5,147921.251991
6,LOT Polish Airlines,CaptMax,Major/National/Low Cost,Poland,16850.0,38.6,66845.127073
7,LOT Polish Airlines,CaptMin,Major/National/Low Cost,Poland,16850.0,38.6,60223.546741
10,Eurowings GmbH,CaptMax,Major/National/Low Cost,Germany,51660.0,62.9,202851.409070
11,Eurowings GmbH,CaptMin,Major/National/Low Cost,Germany,51660.0,62.9,120139.082655


In [80]:
df_model.to_csv("df_model.csv", index=False)